In [21]:
# To default to float division
from __future__ import division

import csv
import pandas
import sklearn
import sklearn.ensemble
from sklearn import feature_extraction
from  sklearn import preprocessing
import scipy
from scipy import sparse
import numpy

print('Done')

Done


In [31]:
data = pandas.read_csv('data/train.csv')

X_train_index, X_test_index, Y_train, Y_test = sklearn.cross_validation.train_test_split(
    data.index, data['LogSalaryNormalized'], test_size=.33, random_state=42)

# Keep train and test as pandas dataframes.
X_train = data.iloc[X_train_index]
X_test = data.iloc[X_test_index]

data.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,TitleRaw,DescriptionLength,LogSalaryNormalized,FullDescriptionWithTitle
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,unknown_value,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,Engineering Systems Analyst,348,10.126631,Engineering Systems Analyst Engineering System...
1,12613049,Engineering Systems Analyst Mathematical Modeller,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,unknown_value,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk,Engineering Systems Analyst / Mathematical Mod...,397,10.221941,Engineering Systems Analyst Mathematical Model...
2,12613647,Pioneer Miser Engineering Systems Analyst,"Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,unknown_value,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,"Pioneer, Miser Engineering Systems Analyst",216,10.126631,Pioneer Miser Engineering Systems Analyst Pion...
3,13179816,Engineering Systems Analyst Water Industry,Engineering Systems Analyst Water Industry Loc...,"Dorking, Surrey, Surrey, Surrey",Dorking,unknown_value,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20K to 30K,25000,cv-library.co.uk,Engineering Systems Analyst Water Industry,1602,10.126631,Engineering Systems Analyst Water Industry Eng...
4,14131336,Senior Subsea Pipeline Integrity Engineer,A globally renowned engineering and training c...,"Aberdeen, Borders",UK,unknown_value,permanent,Indigo 21 Ltd,Engineering Jobs,50000 - 100000/annum,75000,cv-library.co.uk,Senior Subsea Pipeline Integrity Engineer,1364,11.225243,Senior Subsea Pipeline Integrity Engineer A gl...


In [32]:
MIN_WORD_FREQUENCY = 2

# Create function to hash vectorize a field
# Note, will only work with X_train and X_test datasets
def hash_vect(field):
    hash_vect_fn = sklearn.feature_extraction.text.HashingVectorizer(
        stop_words='english', n_features=250,  decode_error='ignore')
    
    X_train_full_description_hash = hash_vect_fn.fit_transform(X_train[field])
    X_test_full_description_hash = hash_vect_fn.transform(X_test[field])
    
    return X_train_full_description_hash, X_test_full_description_hash

# Create function to binarize a field
# Note, will only work with X_train and X_test datasets
def binarize_column(field):
    lb = preprocessing.LabelBinarizer()
    train_output = lb.fit_transform(X_train[field])
    test_output = lb.transform(X_test[field])
    return train_output, test_output

In [33]:
# Hash Vectorize Title, FullDescription
train_title, test_title = hash_vect('Title')
train_full_description, test_full_description = hash_vect('FullDescription')

# Binarize ContractType, ContractTime, Category,
# Company, LocationNormalized
train_contract_type, test_contract_type = binarize_column('ContractType')
train_contract_time, test_contract_time = binarize_column('ContractTime')
train_category, test_category = binarize_column('Category')

train_company, test_company = binarize_column('Company')
train_location, test_location = binarize_column('LocationNormalized')

# Concatenate categorical fields
X_train_categorical = numpy.concatenate((train_contract_type,
    train_contract_time, train_category, train_company, train_location), axis=1)
X_test_categorical = numpy.concatenate((test_contract_type,
    test_contract_time, test_category, test_company, test_location), axis=1)

# Concatenate hash vector fields
X_train_hash_vectors = sparse.hstack((train_title, train_full_description))
X_test_hash_vectors = sparse.hstack((test_title, test_full_description))

# Concatenate all categorical and hash vector datasets into train and test sets
X_training_set = sparse.hstack((X_train_categorical, X_train_hash_vectors))
X_testing_set = sparse.hstack((X_test_categorical, X_test_hash_vectors))

print(X_training_set.shape)
print(X_testing_set.shape)

(114828, 17518)
(56558, 17518)
